In [ ]:
#Import Libraries
import tensorflow as tf
import numpy as np
import collections
import math
import os
import random
import time
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets


In [ ]:
# Load the CIFAR-10 and CIFAR-100 datasets
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
(x_train100, y_train100), (x_test100, y_test100) = datasets.cifar100.load_data()


In [ ]:
#Preprocess the data
def preprocess_images(images):
    images = images.astype('float32')
    images = (images - 127.5) / 127.5  # Normalize the pixel values
    return tf.convert_to_tensor(images)

x_train = preprocess_images(x_train)
x_test = preprocess_images(x_test)

x_train100 = preprocess_images(x_train100)
x_test100 = preprocess_images(x_test100)


In [ ]:
#Define the CNN model architecture
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
# Define the Federated Averaging algorithm for aggregating model updates
def federated_averaging(global_weights, client_weights, num_clients):
    total_weights = np.zeros_like(global_weights)
    for client_weight in client_weights:
        total_weights += client_weight
    global_weights = total_weights / num_clients
    return global_weights

In [ ]:
# Define the centralized server
def server(model, num_clients, rounds, train_data):
    for i in range(rounds):
        # Initialize the client weights
        client_weights = [model.get_weights() for i in range(num_clients)]

        # Send the client weights to each client
        for j in range(num_clients):
            # Train the client model
            client_model = create_model()
            client_model.set_weights(client_weights[j])
            client_model.fit(train_data[j][0], train_data[j][1], epochs=1, verbose=0)

            # Compute the client model weights
            client_weights[j] = client_model.get_weights()

        # Aggregate the client weights using Federated Averaging
        model.set_weights(federated_averaging(model.get_weights(), client_weights, num_clients))

    return model

In [ ]:
# Define the clients
def clients(model, num_clients, train_data):
    for i in range(num_clients):
        # Train the client model
        client_model = create_model()
        client_model.set_weights(model.get_weights())
        client_model.fit(train_data[i][0], train_data[i][1], epochs=1, verbose=0)

        # Compute the client model weights
        client_weights = client_model.get_weights()

        # Send the client weights back to the server
        yield client_weights

In [ ]:
# Load and preprocess the CIFAR-10 and CIFAR-100 datasets
cifar10 = tf.keras.datasets.cifar10.load_data()
cifar100 = tf.keras.datasets.cifar100.load_data()

cifar10_train, cifar10_test = cifar10
cifar100_train, cifar100_test = cifar100

cifar10_train = (cifar10_train[0] / 255.0, tf.keras.utils.to_categorical(cifar10_train[1]))
cifar10_test = (cifar10_test[0] / 255.0, tf.keras.utils.to_categorical(cifar10_test[1]))
cifar100_train = (cifar100_train[0] / 255.0, tf.keras.utils.to_categorical(cifar100_train[1]))
cifar100_test = (cifar100_test[0] / 255.0, tf.keras.utils.to_categorical(cifar100_test[1]))


In [ ]:
# Define the number of clients and training rounds
NUM_CLIENTS = 500
NUM_CLIENTS_TEST = 100
NUM_ROUNDS = 50
NUM_CLIENTS_PER_ROUND = 10

In [ ]:
# Load the CIFAR-10 and CIFAR-100 datasets
cifar10_train, cifar10_test = tf.keras.datasets.cifar10.load_data()
cifar100_train, cifar100_test = tf.keras.datasets.cifar100.load_data()


In [ ]:
# Define a function to preprocess the data
def preprocess_data(raw_x, raw_y):
    x = tf.cast(raw_x, tf.float32) / 255.0
    y = tf.cast(raw_y, tf.int32)
    return {'image': x}, y

# Create the preprocessed datasets
preprocessed_train_data = {
    'cifar10': tf.data.Dataset.from_tensor_slices(cifar10_train).map(preprocess_data),
    'cifar100': tf.data.Dataset.from_tensor_slices(cifar100_train).map(preprocess_data)
}

preprocessed_test_data = {
    'cifar10': tf.data.Dataset.from_tensor_slices(cifar10_test).map(preprocess_data),
    'cifar100': tf.data.Dataset.from_tensor_slices(cifar100_test).map(preprocess_data)
}


In [ ]:
# Define a function to evaluate the model on a dataset
def evaluate_model(model, dataset):
    loss = tf.keras.metrics.Mean()
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for x, y in dataset:
        y_pred = model(x)['output']
        loss(y, y_pred)
        accuracy(y, y_pred)

    return {'loss': loss.result().numpy(), 'accuracy': accuracy.result().numpy()}


In [ ]:
# Define a function to create the model for each client
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=preprocessed_train_data['cifar10'].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

# Define the federated averaging process
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.1),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)


In [ ]:
def train_federated_model(train_data, test_data, num_clients, num_rounds, batch_size, learning_rate):
# Create the centralized model
  global_model = create_cifar10_model() if isinstance(train_data.element_spec, tuple) else create_cifar100_model()
  global_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


In [ ]:


# Initialize the server state
state = fed_avg.initialize()

# Train the model using federated learning
for round_num in range(NUM_ROUNDS):
    # Sample a subset of clients to participate in this round
    sampled_clients = np.random.choice(client_ids, NUM_CLIENTS_PER_ROUND, replace=False)

    # Create a list of federated datasets for the selected clients
    federated_train_data = [preprocessed_train_data.create_tf_dataset_for_client(client) for client in sampled_clients]

    # Train on the selected clients' data
    state, metrics = fed_avg.next(state, federated_train_data)

    # Evaluate the model on the test data after every 10 rounds
    if (round_num + 1) % 10 == 0:
        test_metrics = evaluate_model(model, preprocessed_test_data)
        print(f"Round {round_num + 1}, Test metrics: {test_metrics}")


In [ ]:
#Partition the data across clients
def partition_data(images, labels, num_clients):
    """Partition the data across the given number of clients."""
    num_items_per_client = len(images) // num_clients
    partitioned_images = collections.defaultdict(list)
    partitioned_labels = collections.defaultdict(list)
    for i in range(num_clients):
        for j in range(num_items_per_client):
            idx = i * num_items_per_client + j
            partitioned_images[i].append(images[idx])
            partitioned_labels[i].append(labels[idx])
    return partitioned_images, partitioned_labels

num_clients = 500
x_train_partitions, y_train_partitions = partition_data(x_train, y_train, num_clients)

num_clients100 = 100
x_train_partitions100, y_train_partitions100 = partition_data(x_train100, y_train100, num_clients100)


In [ ]:
#Define the federated learning algorithm